In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Создание датасета

In [ ]:
image_dir = 'datatset_buildings\\augment_blur_img' #path to image directories 
mask_dir = 'datatset_buildings\\augment_mask' #path to masks dorectories
 
IMG_HEIGHT = 256
IMG_WIDTH = 256

def load_data(image_dir, mask_dir):
    images = []
    masks = []

    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename.replace('.jpg', '.png'))  

        image = Image.open(img_path).convert('RGB').resize((IMG_WIDTH, IMG_HEIGHT))
        mask = Image.open(mask_path).resize((IMG_WIDTH, IMG_HEIGHT))

        image = np.array(image) / 255.0  
        mask = np.array(mask)

        mask = np.where(mask > 0, 1, 0) 

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

images, masks = load_data(image_dir, mask_dir)

masks = np.expand_dims(masks, axis=-1)

print("Image shape:", images.shape) 
print("Mask shape:", masks.shape) 

# Инициализация модели и обучение

In [17]:
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

def unet_plus_plus(input_size=(IMG_HEIGHT, IMG_WIDTH, 3), n_classes=1):
    inputs = layers.Input(input_size)

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    x1_0 = c1  
    x2_0 = c2 
    x3_0 = c3 
    x4_0 = c4 
    x5_0 = c5  

    x4_1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x4_0, layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(x5_0)], axis=-1)
    )

    x3_1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x3_0, layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4_1)], axis=-1)
    )
    x3_2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x3_1, layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x4_1)], axis=-1)
    )

    x2_1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x2_0, layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x3_1)], axis=-1)
    )
    x2_2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x2_1, layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x3_2)], axis=-1)
    )

    x1_1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x1_0, layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x2_1)], axis=-1)
    )
    x1_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(
        layers.concatenate([x1_1, layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x2_2)], axis=-1)
    )

    outputs = layers.Conv2D(n_classes, (1, 1), activation='sigmoid')(x1_2)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [5]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def focal_dice_loss(y_true, y_pred, alpha=0.25, gamma=2):
    dice = dice_coefficient(y_true, y_pred)
    focal_dice = alpha * K.pow(1 - dice, gamma)
    return 1 - dice + focal_dice

early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-4, restore_best_weights=True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=2, min_lr=1e-6)

model = unet_plus_plus()
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss=focal_dice_loss, 
              metrics=['accuracy', dice_coefficient, iou])

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=100,  
    callbacks=[early_stopping, lr_reducer]
)

In [ ]:
model.save('unet_buildings_median_blur_binary_256_250_new.h5')

# Загрузка модели и проверка на валидационном датасете

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('models\\unet++_buildings_median_blur_binary_256_250.h5', custom_objects={'focal_dice_loss': focal_dice_loss, 'dice_coefficient': dice_coefficient, 'iou': iou})

In [ ]:
def plot_sample_predictions(X_val, y_val, model, num_samples=5):
    preds = model.predict(X_val)
    
    for i in range(len(preds)):
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].imshow(X_val[i])
        ax[0].set_title("Init image")

        ax[1].imshow(y_val[i], cmap='gray')
        ax[1].set_title("Init mask")
        
        binary_array = np.where(preds[i] > 0.8, 1, 0).astype(np.uint8)
        ax[2].imshow(binary_array, cmap='gray')
        
        ax[2].imshow(preds[i])
        ax[2].set_title(f"Predicted mask DICE: {dice_coefficient(y_val[i], preds[i])}")
        print(1-focal_dice_loss(y_val[i],preds[i]))
        
        plt.show()

plot_sample_predictions(X_val, y_val, model)